In [1]:
import pandas as pd
import numpy as np

In [2]:
example_fire = pd.read_csv('Fire hotspot/viirs-snpp_2016_Cambodia.csv')

example_fire['hour'] = example_fire['acq_time'].apply(str).str[:-2]

example_fire['datetime'] = pd.to_datetime(
                                            example_fire['acq_date']+'T'+example_fire['hour'].str.zfill(2), 
                                            format='%Y-%m-%dT%H')

In [3]:
filtered = example_fire[['latitude','longitude','bright_ti4','bright_ti5','frp','datetime']]
filtered.head(3)

,latitude,longitude,bright_ti4,bright_ti5,frp,datetime
0,11.128095,103.598412,340.71,300.81,4.28,2016-01-01 06:00:00
1,11.331814,105.105881,342.53,298.30,6.83,2016-01-01 06:00:00
2,10.649994,103.775459,332.40,299.12,4.63,2016-01-01 06:00:00


In [4]:
fire_Cambodia_2016 = filtered.groupby('datetime').mean()
fire_Cambodia_2016.head(3)

,latitude,longitude,bright_ti4,bright_ti5,frp
datetime,,,,,
2016-01-01 06:00:00,13.123613,105.508592,342.235212,299.471295,7.933461
2016-01-01 17:00:00,12.971076,106.365074,307.710966,289.430966,1.661250
2016-01-01 19:00:00,13.303865,105.259085,305.723942,288.617404,1.588750


In [9]:
years = ['2016','2017','2018','2019']
countries = ['Cambodia','Myanmar','Thailand','Lao_PDR']

fire = {'2016':{}, '2017':{}, '2018':{}, '2019':{}}

In [13]:
for year in years:
    for country in countries:
        df = pd.read_csv(f'./Fire hotspot/viirs-snpp_{year}_{country}.csv')
        df['hour'] = df['acq_time'].apply(str).str[:-2]
        
        df['datetime'] = pd.to_datetime(df['acq_date']+'T'+df['hour'].str.zfill(2), 
                            format='%Y-%m-%dT%H')   
        fire[year][country] = df[['latitude','longitude','bright_ti4','bright_ti5','frp','datetime']]

In [16]:
fire[year][country].groupby('datetime').mean()

,latitude,longitude,bright_ti4,bright_ti5,frp
datetime,,,,,
2019-01-01 05:00:00,15.784060,105.414863,335.452405,297.472658,4.922785
2019-01-01 18:00:00,17.045321,104.933214,302.311429,281.548571,0.575714
2019-01-02 05:00:00,17.282650,104.869486,337.637419,293.852581,5.232903
2019-01-02 07:00:00,18.924614,101.750069,337.266667,289.783333,6.006667
2019-01-02 18:00:00,15.789707,105.181188,310.707333,281.984000,1.951333
...,...,...,...,...,...
2019-12-30 05:00:00,16.466402,105.250331,338.574000,300.210462,9.720154
2019-12-30 18:00:00,16.778148,105.091142,306.907778,289.616667,0.863333
2019-12-31 05:00:00,16.147016,105.136903,342.156786,292.761786,5.227143
